In [1]:
library(tidyverse)
library(MASS)
library(DescTools)
library(pwr)
library(repr)
library(cowplot)
library(ggseqlogo)
library(reshape2)

── Attaching core tidyverse packages ──────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.0     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.1     ✔ tibble    3.1.8
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select



Attaching package: ‘cowplot’


The following object is masked from ‘package:lubridate’:

    stamp



Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




In [2]:
cols = c('chromosome', 'reference_nucleotide', 'position', 'trinuc_context','dinuc_context', 
         "methylation_frequency", "count_mC", "coverage")

TS559statS <- read.delim('../processed_cgmaps/bs7v1/TS559statS_coverage', sep='\t') %>%
    dplyr::rename(
          methylation_frequency_TS559_rep1 = methylation_frequency_TS559_March2016,
          mC_count_TS559_rep1 = mC_count_TS559_March2016,
          coverage_TS559_rep1 = coverage_TS559_March2016,
        
          methylation_frequency_TS559_rep2 = methylation_frequency_TS559_Dec2016,
          mC_count_TS559_rep2 = mC_count_TS559_Dec2016,
          coverage_TS559_rep2 = coverage_TS559_Dec2016,
           
          methylation_frequency_TS559_rep3 = methylation_frequency_TS559_Oct2020,
          mC_count_TS559_rep3 = mC_count_TS559_Oct2020,
          coverage_TS559_rep3 = coverage_TS559_Oct2020
          )

annotation <- data.frame(read.delim('/home/kristin/jupyter_notebooks/RNA_mt_proj/bs7/annotation_ready', sep = '\t') %>%
    dplyr::rename(position=TS559_position) )

In [3]:
#####HELPERS 

load_cgmap <- function(cgmap,strain, replicate, growth_phase, metabolic_condition='S'){
    cols = c('chromosome', 'base', 'position', 'trinuc_context','dinuc_context', 
         "methylation_frequency", "count_mC", "coverage")
    outdata <- read.delim(cgmap, sep='\t', header=FALSE,col.names=cols) %>%
        mutate(strain=strain, growth_phase=growth_phase, 
               metabolic_condition=metabolic_condition, replicate=replicate)

}


#clean up raw CGmaps, add some and rename some columns
processCGmap <- function(cgmap){
    strain = unique(cgmap$strain)
    rep = unique(cgmap$replicate)
    
    final_data <- cgmap %>% 
        dplyr::select(-trinuc_context, -dinuc_context) %>%
        dplyr::rename(!!paste("coverage","_",strain,"_",rep, sep="") := coverage) %>%
        dplyr::rename(!!paste("mC_count","_",strain,"_",rep, sep="") := count_mC) %>%
        dplyr::rename(!!paste("methylation_frequency","_",strain,"_",rep, sep="") := methylation_frequency) %>%
        dplyr::select(-replicate, -strain)
    return(final_data)
}



#merge data for replicate deletion strains
merge_del_reps <- function(x,y, strain){
    cov_1 <- paste("coverage_",strain, "_rep1", sep='')
    cov_2 <- paste("coverage_",strain, "_rep2", sep='')

    d <- x %>%
        merge(y=y, 
              by=c("chromosome","position", 'reference_nucleotide', 'growth_phase','metabolic_condition'), 
              all=TRUE) %>%
    
            mutate(!!as.name(cov_1) := replace_na(!!as.name(cov_1),0)) %>%
            mutate(!!as.name(cov_2) := replace_na(!!as.name(cov_2),0))
    
    return(d)
}

#merge data for triplicate deletion strains
merge_del_3reps <- function(x,y,z,strain){
    cov_1 <- paste("coverage_",strain, "_rep1", sep='')
    cov_2 <- paste("coverage_",strain, "_rep2", sep='')
    cov_3 <- paste("coverage_",strain, "_rep3", sep='')

    d <- x %>%
        merge(y=y, 
              by=c("chromosome","position", 'reference_nucleotide', 'growth_phase','metabolic_condition'), 
              all=TRUE) %>%
        merge(y=z, 
              by=c("chromosome","position", 'reference_nucleotide', 'growth_phase','metabolic_condition'), 
              all=TRUE) %>%
            mutate(!!as.name(cov_1) := replace_na(!!as.name(cov_1),0)) %>%
            mutate(!!as.name(cov_2) := replace_na(!!as.name(cov_2),0)) %>%
            mutate(!!as.name(cov_3) := replace_na(!!as.name(cov_3),0))

    return(d)
}


# determine if high confidence, reproducible sites with 3 reps
#f  frequency
#m  m5C coverage
#p  99 percentile m5C coverage
#c  total coverage
detect_hiconf_3reps <- function(f1,m1,p1,c1,f2,m2,p2,c2,f3,m3,p3,c3, min_cov=47,
                                min_freq=.10, min_freq2 = 0.05) {
    ls <- c()
    
    if( is.na(c1) ) {c1<-0}
    if( is.na(c2) ) {c2<-0}
    if( is.na(c3) ) {c3<-0}
    if(c1 < min_cov & c2 < min_cov & c3 < min_cov) {return(NA)}
    
    else{
    
    if(c1<min_cov){ls <- c(ls,NA)}
    else if(  (c1>=min_cov & m1>=p1 & f1>=min_freq) | (m1>50 & f1>=min_freq2) ) {ls <- c(ls,TRUE)}
    else {ls <- c(ls,FALSE)}
    
    if(c2<min_cov){ls <- c(ls,NA)}
    else if( (c2>=min_cov & m2>=p2 & f2>=min_freq) | (m2>50 & f2>=min_freq2) ) {ls <- c(ls,TRUE)}
    else {ls <- c(ls,FALSE)}
    
    if(c3<min_cov){ls <- c(ls,NA)}
    else if( (c3>=min_cov & m3>=p3 & f3>=min_freq)| (m3>50 & f3>=min_freq2) )  {ls <- c(ls,TRUE)}
    else {ls <- c(ls,FALSE)}
    
    ans_t <- length(which(ls==TRUE))
    ans_na <- length(which(ls==NA))
    ans_f <- length(which(ls==FALSE))

    if(ans_na >= 3){return(NA)}
    if(ans_na == 2 & ans_t == 1){return(NA)}
    else if(ans_t >= 2){return(TRUE)}
    else { return(FALSE)}
        }
}



# determine if high confidence, reproducible sites with 2 reps
#f  frequency
#m  m5C coverage
#p  99 percentile m5C coverage
#c  total coverage
detect_hiconf_2reps <- function(f1,m1,p1,c1,f2,m2,p2,c2, min_cov=47,
                                min_freq=.10, min_freq2 = 0.05) {
    ls <- c()
    
    if( is.na(c2) ) {c1<-0}
    if( is.na(c2) ) {c2<-0}
    
    if(c1 < min_cov & c2 < min_cov) {return(NA)}
    else{
            
    if(c1<min_cov){ls <- c(ls,NA)}
    else if(  (c1>=min_cov & m1>=p1 & f1>=min_freq) | (m1>50 & f1>=min_freq2) ) {ls <- c(ls,TRUE)}
    else {ls <- c(ls,FALSE)}
    
    if(c2<min_cov){ls <- c(ls,NA)}
    else if( (c2>=min_cov & m2>=p2 & f2>=min_freq) | (m2>50 & f2>=min_freq2) ) {ls <- c(ls,TRUE)}
    else {ls <- c(ls,FALSE)}

    ans_t <- length(which(ls==TRUE))
    ans_na <- length(which(ls==NA))
    ans_f <- length(which(ls==FALSE))

    if(ans_f == 1 & ans_t == 1 ){return(TRUE)}
    if(ans_f >= 1  ){return(FALSE)}
    else if(ans_t >= 2){return(TRUE)}
    else if(ans_t == 1 & ans_na ==1){return(TRUE)}
    else if(ans_f == 1 & ans_na ==1){return(FALSE)}

    else { return(NA)}
        
        }
}



# determine if high confidence in all three 3 reps
super_hiconf_3reps <- function(f1,m1,p1,c1,f2,m2,p2,c2,f3,m3,p3,c3, min_cov=47,
                                min_freq=.10, min_freq2 = 0.05) {
    ls <- c()
    
    if( is.na(c1) ) {c1<-0}
    if( is.na(c2) ) {c2<-0}
    if( is.na(c3) ) {c3<-0}

    
    if(  (c1>=min_cov & m1>=p1 & f1>=min_freq) | (c1>1000 & f1>=min_freq2)  ) {ls <- c(ls,TRUE)}
    else {ls <- c(ls,FALSE)}
    
    
    if( (c2>=min_cov & m2>=p2 & f2>=min_freq) | (c2>=1000 & f2>=min_freq2) ) {ls <- c(ls,TRUE)}
    else {ls <- c(ls,FALSE)}
    
    if( (c3>=min_cov & m3>=p3 & f3>=min_freq) | (c3>=1000 & f3>=min_freq2) )  {ls <- c(ls,TRUE)}
    else {ls <- c(ls,FALSE)}
    
    ans <- length(which(ls==TRUE))
    
    if(ans >= 3){return(TRUE)}
    else { return(FALSE)}
}



#determine if the sample m5C frequency are within a 90% confidence interval
rm_overlap <- function(sp, ssd, cp,csd){
    ssd <- ssd*1.645
    csd <- csd*1.645

    r1 <- c(sp-ssd, sp+ssd)
    r2 <- c(cp-csd, cp+csd)
    ans <- r1 %overlaps% r2
    return(ans)
    
}


# get p value of each m5C site using binomial distribution
#glm binomial

get_pvalue <- function(s1_mC,s1_cov, s2_mC,s2_cov,
                       c1_mC,c1_cov,c2_mC,c2_cov,c3_mC,c3_cov){

    if(s1_cov == 0 & s2_cov == 0){return(NA)} 
    if(c1_cov == 0 & c2_cov == 0 & c3_cov == 0){return(NA)} 

    else{

    sample_success = s1_mC + s2_mC  + 1
    sample_failure = (s1_cov+s2_cov)-sample_success  +1
    
    control_success = c1_mC + c2_mC + c3_mC  +1
    control_failure = (c1_cov + c2_cov + c3_cov) - control_success  +1
    
    if(is.na(sample_failure) |  is.na(control_failure) |
      is.na(sample_failure) | is.na(sample_failure)
      
      ){return(NA)}
    
    treatment <- c('sample','control')
    success <- c(sample_success, control_success)
    failure <- c(sample_failure, control_failure)
        
    da <- data.frame(treatment, success, failure)
        da$success <- as.integer(da$success)
        da$failure <- as.integer(da$failure)
        da$treatment <- as.factor(da$treatment)

        table <- cbind(da$success,da$failure)

    fit <- glm(table ~ treatment, family = binomial, data = da)
    p_val <- summary(fit)$coefficients[8]
    return(p_val) 
    }
}

get_pvalue_deprecated <- function(s1_mC,s1_cov, s2_mC,s2_cov,
                       c1_mC,c1_cov,c2_mC,c2_cov,c3_mC,c3_cov){
    
    meth_freq_r1 <- (s1_mC+1)/(s1_cov+1)
    meth_freq_r2 <- (s2_mC+1)/(s2_cov+1)
    cnt_meth_freq_r1 <- (c1_mC+1)/(c1_cov+1)
    cnt_meth_freq_r2 <- (c2_mC+1)/(c2_cov+1)
    cnt_meth_freq_r3 <- (c3_mC+1)/(c3_cov+1)
    
    
    strain <- c('trt','trt','cnt','cnt','cnt')
    freq <- c(meth_freq_r1, meth_freq_r2,
              cnt_meth_freq_r1, cnt_meth_freq_r2, cnt_meth_freq_r3)
    
    
    da <- data.frame(cbind(strain, freq))
    
    da$strain <- as.factor(da$strain)
    da$freq <- as.double(da$freq)

    pval <- round(summary(aov(freq ~ strain, data=da))[[1]][1,5], 4)    
    return(pval)

}

    
get_pvalue_3reps <- function(s1_mC,s1_cov, s2_mC,s2_cov,s3_mC,s3_cov,
                       c1_mC,c1_cov,c2_mC,c2_cov,c3_mC,c3_cov){
    
    
    if(s1_cov == 0 & s2_cov==0 & s3_cov == 0){return(NA)} 
    if(c1_cov == 0 & c2_cov==0 & c3_cov == 0){return(NA)} 

    sample_success = s1_mC + s2_mC + s3_mC +1
    sample_failure = (s1_cov + s2_cov + s3_cov)-sample_success +1
    
    control_success = c1_mC + c2_mC + c3_mC +1
    control_failure = (c1_cov + c2_cov + c3_cov) - control_success +1
        
    if(is.na(sample_failure) |  is.na(control_failure) |
      is.na(sample_failure) | is.na(sample_failure) ){return(NA)}
    
    treatment <- c('sample','control')
    success <- c(sample_success, control_success)
    failure <- c(sample_failure,control_failure)
        
    da <- data.frame(treatment, success, failure)
        da$success <- as.integer(da$success)
        da$failure <- as.integer(da$failure)
        da$treatment <- as.factor(da$treatment)

        table <- cbind(da$success,da$failure)

    fit <- glm(table ~ treatment, family = binomial, data = da)
    p_val <- summary(fit)$coefficients[8]
    return(p_val)
}
    

get_quantiles <- function(strain_cgmap, strain, reps = 2, min_cov=47){
    
    m5C_count_r1 <- paste("mC_count_", strain, "_rep1", sep="")
    cov_r1 <- paste("coverage_", strain, "_rep1", sep="")

    m5C_count_r2 <- paste("mC_count_", strain, "_rep2", sep="")
    cov_r2 <- paste("coverage_", strain, "_rep2", sep="")
    
    #get quantile values for m5C count
    q_strain_rep1 <- quantile(
        (strain_cgmap %>%
        filter(!!as.name(cov_r1)>=min_cov, !!as.name(m5C_count_r1)>=0) )[[m5C_count_r1]], .99)[[1]]
    
    q_strain_rep2 <- quantile(
        (strain_cgmap %>%
        filter(!!as.name(cov_r2)>=min_cov, !!as.name(m5C_count_r2)>=0) )[[m5C_count_r2]], .99)[[1]]
    
    q <- c(q_strain_rep1, q_strain_rep2)
    
    if(reps==3){
        m5C_count_r3 <- paste("mC_count_", strain, "_rep3", sep="")
        cov_r3 <- paste("coverage_", strain, "_rep3", sep="")
        
        q_strain_rep3 <- quantile(
        (strain_cgmap %>%
        filter(!!as.name(cov_r3)>=min_cov, !!as.name(m5C_count_r3)>=0) )[[m5C_count_r3]], .99)[[1]]
        
        q = c(q, q_strain_rep3)
    }
    
    return(q)
}
    

get_difference <- function(hiconf_TS559, hiconf_strain, strain_poolf, control_poolf, CI90_overlap){
    
    if(is.na(hiconf_TS559) | is.na(hiconf_strain)){return(NA)}
    else if(is.na(CI90_overlap)){return(NA)}
    
    else if(hiconf_TS559 == FALSE & hiconf_strain == FALSE){return("NOT MODIFIED")}
    else if(CI90_overlap == TRUE){return("OUTSIDE OF DIFFERENTIATION LIMITS")}

    else if(hiconf_TS559 == TRUE & hiconf_strain == FALSE & strain_poolf <= 0.02){return("ABS LOSS")} 
    else if(hiconf_TS559 == TRUE & hiconf_strain == FALSE & strain_poolf > 0.02){return("REL LOSS")} 

    else if(hiconf_TS559 == FALSE & hiconf_strain == TRUE & control_poolf <= 0.02){return("ABS GAIN")}
    else if(hiconf_TS559 == FALSE & hiconf_strain == TRUE & control_poolf > 0.02){return("REL GAIN")}

    else if(hiconf_TS559 == TRUE & hiconf_strain == TRUE & strain_poolf > control_poolf){return("REL GAIN")} 
    else if(hiconf_TS559 == TRUE & hiconf_strain == TRUE & strain_poolf < control_poolf){return("REL LOSS")} 
    
    else{return("error")}
    }

        
CompareCGmap <- function(strain_cgmap, TS559_cgmap, strain, annotation, minimum_freq=.10, min_cov=47){
    
    #making some column names
    meth_freq_r1 <- paste("methylation_frequency_", strain, "_rep1", sep="")
    m5C_count_r1 <- paste("mC_count_", strain, "_rep1", sep="")
    cov_r1 <- paste("coverage_", strain, "_rep1", sep="")

    meth_freq_r2 <- paste("methylation_frequency_", strain, "_rep2", sep="")
    m5C_count_r2 <- paste("mC_count_", strain, "_rep2", sep="")
    cov_r2 <- paste("coverage_", strain, "_rep2", sep="")
    
    pooled_methylation_frequency_strain <- paste("pooled_methylation_frequency_", strain, sep='')

    #get quantile values for m5C count
    q_TS559_rep1 <- quantile((TS559_cgmap%>%
        filter(coverage_TS559_rep1>=min_cov, mC_count_TS559_rep1>=0))$mC_count_TS559_rep1, .99)[[1]]
    
    q_TS559_rep2 <- quantile((TS559_cgmap%>%
        filter(coverage_TS559_rep2>=min_cov, mC_count_TS559_rep2>=0))$mC_count_TS559_rep2, .99)[[1]]
    
    q_TS559_rep3 <- quantile((TS559_cgmap%>%
        filter(coverage_TS559_rep3>=min_cov, mC_count_TS559_rep3>=0))$mC_count_TS559_rep3, .99)[[1]]

    q_strain_rep1 <- quantile(
        (strain_cgmap %>%
        filter(!!as.name(cov_r1)>=min_cov, !!as.name(m5C_count_r1)>=0) )[[m5C_count_r1]], .99)[[1]]
    
    q_strain_rep2 <- quantile(
        (strain_cgmap %>%
        filter(!!as.name(cov_r2)>=min_cov, !!as.name(m5C_count_r2)>=0) )[[m5C_count_r2]], .99)[[1]]
    
    
    #build dataframe
    data <- strain_cgmap %>%
        merge(y = TS559_cgmap, all = TRUE, 
             by= c("chromosome","position","reference_nucleotide","growth_phase","metabolic_condition")) %>%
        dplyr::select(-chromosome,-standard_devition) %>%
    
        # house keeping
        mutate(!!as.name(cov_r1) := replace_na(!!as.name(cov_r1),0)) %>%
        mutate(!!as.name(cov_r2) := replace_na(!!as.name(cov_r2),0)) %>%
        
        mutate(coverage_TS559_rep1 = replace_na(coverage_TS559_rep1,0)) %>%
        mutate(coverage_TS559_rep2 = replace_na(coverage_TS559_rep2,0)) %>%
        mutate(coverage_TS559_rep3 = replace_na(coverage_TS559_rep3,0)) %>%
    rowwise() %>%
        mutate(hiconf_TS559 = detect_hiconf_3reps(
            f1=methylation_frequency_TS559_rep1,
            m1=mC_count_TS559_rep1,
            p1=q_TS559_rep1,
            c1=coverage_TS559_rep1,
            f2=methylation_frequency_TS559_rep2,
            m2=mC_count_TS559_rep2,
            p2=q_TS559_rep2,
            c2=coverage_TS559_rep2,
            f3=methylation_frequency_TS559_rep3,
            m3=mC_count_TS559_rep3,
            p3=q_TS559_rep3,
            c3=coverage_TS559_rep3,
            min_cov=min_cov,
            min_freq = minimum_freq) 
              ) %>%
        
        rowwise() %>%
        mutate(hiconf_strain = detect_hiconf_2reps(
                f1=!!as.name(meth_freq_r1),
                m1=!!as.name(m5C_count_r1),
                p1=q_strain_rep1,
                c1=!!as.name(cov_r1),
                f2=!!as.name(meth_freq_r2),
                m2=!!as.name(m5C_count_r2),
                p2=q_strain_rep2,
                c2=!!as.name(cov_r2), 
                min_cov= min_cov,
                min_freq = minimum_freq)
                 
                ) %>%
    
    # keep sites where m5C is high confidence in either strain
    filter(hiconf_TS559 == TRUE | hiconf_strain == TRUE) %>%
    filter(!is.na(hiconf_TS559) & !is.na(hiconf_strain)) %>%

    # calculate the mean m5C frequency at each site by pooling replicates (round to 2 decimals)
    # calculate standard deviation of each site as well
    mutate(!!as.name(pooled_methylation_frequency_strain) := 
             ( !!as.name(m5C_count_r1) +  !!as.name(m5C_count_r2) ) / ( !!as.name(cov_r1) + !!as.name(cov_r2) ) ) %>%
        
    mutate(!!as.name(pooled_methylation_frequency_strain) := round(!!as.name(pooled_methylation_frequency_strain),2))%>%
    mutate(strain_sd = sd(c(!!as.name(meth_freq_r1),!!as.name(meth_freq_r2))))%>%
    
    mutate(pooled_methylation_frequency_TS559 =
               (mC_count_TS559_rep1 + mC_count_TS559_rep2 + mC_count_TS559_rep3) /
               (coverage_TS559_rep1+coverage_TS559_rep2+coverage_TS559_rep3)    ) %>%
    mutate(pooled_methylation_frequency_TS559=round(pooled_methylation_frequency_TS559,2)) %>%
    mutate(TS559_sd = sd(c(methylation_frequency_TS559_rep1,methylation_frequency_TS559_rep2,
                              methylation_frequency_TS559_rep3))) %>%

    # calculate fold change of methylaton frequency at each site between parent and deletion strain
    mutate(FoldChange = (!!as.name(pooled_methylation_frequency_strain)+.01) / 
                               (pooled_methylation_frequency_TS559+.01),
           log2FC = log2( (!!as.name(pooled_methylation_frequency_strain)+.01) / 
                               (pooled_methylation_frequency_TS559+.01) ) ) %>%
    
    mutate(CI90_overlap = 
                   rm_overlap(!!as.name(pooled_methylation_frequency_strain), strain_sd,
                    pooled_methylation_frequency_TS559, TS559_sd )) %>%
    
    mutate(difference = get_difference(hiconf_TS559, hiconf_strain,   
                                            !!as.name(pooled_methylation_frequency_strain), 
                                            pooled_methylation_frequency_TS559,
                                            CI90_overlap
                                      ) ) %>%
    filter(difference == "ABS LOSS" | difference == "REL LOSS" | difference == "ABS GAIN" | 
           difference == "REL GAIN" | difference == "error") %>%
    filter(log2FC >= 1 | log2FC <= -1) %>%

    # add p vlaue
    rowwise() %>%
    mutate(pvalue = get_pvalue(
            s1_mC = !!as.name(m5C_count_r1),
            s1_cov = !!as.name(cov_r1), 
            s2_mC = !!as.name(m5C_count_r2),
            s2_cov = !!as.name(cov_r2),
            c1_mC = mC_count_TS559_rep1,
            c1_cov = coverage_TS559_rep1,
            c2_mC = mC_count_TS559_rep2,
            c2_cov = coverage_TS559_rep2,
            c3_mC = mC_count_TS559_rep3,
            c3_cov = coverage_TS559_rep3) ) %>%
    
    filter(!is.na(pvalue), pvalue <= 0.01)
    
        
    # adjust p values
    # adj_pvalue <- p.adjust(data$pvalue, method = 'fdr')
    #data <- cbind(data, adj_pvalue)
    
    data <- data %>% merge(y=annotation,by='position', all.x=TRUE) %>%
            dplyr::rename(TS559_position = position) %>%
           dplyr::select(-reproducible, -highly_reproducible,-reference_nucleotide) %>%
           arrange(log2FC) %>%
           arrange(difference) %>%
           dplyr::select(TS559_position, KOD1_position, strand, everything())
    
  
    return(data)
}
    

enumerate_hiconf_2reps <- function(strain_cgmap, strain, min_cov=47){
    
    meth_freq_r1 <- paste("methylation_frequency_", strain, "_rep1", sep="")
    m5C_count_r1 <- paste("mC_count_", strain, "_rep1", sep="")
    cov_r1 <- paste("coverage_", strain, "_rep1", sep="")

    meth_freq_r2 <- paste("methylation_frequency_", strain, "_rep2", sep="")
    m5C_count_r2 <- paste("mC_count_", strain, "_rep2", sep="")
    cov_r2 <- paste("coverage_", strain, "_rep2", sep="")
    
    q_strain_rep1 <- quantile(
        (strain_cgmap %>%
        filter(!!as.name(cov_r1)>=min_cov, !!as.name(m5C_count_r1)>=0) )[[m5C_count_r1]], .99)[[1]]
    
    q_strain_rep2 <- quantile(
        (strain_cgmap %>%
        filter(!!as.name(cov_r2)>=min_cov, !!as.name(m5C_count_r2)>=0) )[[m5C_count_r2]], .99)[[1]]
    
    
    data <- strain_cgmap %>%
        dplyr::select(-chromosome, -reference_nucleotide,-growth_phase) %>%
    
        mutate(!!as.name(cov_r1) := replace_na(!!as.name(cov_r1),0)) %>%
        mutate(!!as.name(cov_r2) := replace_na(!!as.name(cov_r2),0)) %>%  
        
        rowwise() %>%
        mutate(hiconf_strain = detect_hiconf_2reps(
                f1=!!as.name(meth_freq_r1),
                m1=!!as.name(m5C_count_r1),
                p1=q_strain_rep1,
                c1=!!as.name(cov_r1),
                f2=!!as.name(meth_freq_r2),
                m2=!!as.name(m5C_count_r2),
                p2=q_strain_rep2,
                c2=!!as.name(cov_r2), 
                min_cov=min_cov)
                 
                ) %>%
        dplyr::rename(TS559_position = position) %>%
        
        mutate( 'm5C_cov/total_cov_rep1' := 
            paste(!!as.name(paste('mC_count_',strain,'_rep1',sep='')), 
                  '/', 
                  !!as.name(paste('coverage_',strain,'_rep1',sep='')) 
                 ) 
              ) %>%
        
        mutate( 'm5C_cov/total_cov_rep2' := 
            paste(!!as.name(paste('mC_count_',strain,'_rep2',sep='')), 
                  '/', 
                  !!as.name(paste('coverage_',strain,'_rep2',sep='')) 
                 ) 
              ) %>%
        dplyr::select(TS559_position, 
                  !!as.name(meth_freq_r1), 
                  'm5C_cov/total_cov_rep1',
                  !!as.name(meth_freq_r2), 
                  'm5C_cov/total_cov_rep2',
                     hiconf_strain) %>%

    dplyr::rename(!!paste(strain,'_hiconf',sep=''):= hiconf_strain )  %>%
    dplyr::rename(!!paste('m5C_cov/total_cov_',strain,'_rep1',sep=''):= 'm5C_cov/total_cov_rep1' )  %>%
    dplyr::rename(!!paste('m5C_cov/total_cov_',strain,'_rep2',sep=''):= 'm5C_cov/total_cov_rep2' ) %>%
    dplyr::rename(!!paste('freq_',strain,'_rep1',sep=''):= !!as.name(paste('methylation_frequency_',strain,'_rep1',sep='') ) )%>%
    dplyr::rename(!!paste('freq_',strain,'_rep2',sep=''):= !!as.name(paste('methylation_frequency_',strain,'_rep2',sep='') ) )  
    
    return(data)
}

In [4]:
CompareCGmap_3reps <- function(strain_cgmap, TS559_cgmap, strain, annotation, minimum_freq=.10, min_cov=47){
    
    #making some column names
    meth_freq_r1 <- paste("methylation_frequency_", strain, "_rep1", sep="")
    m5C_count_r1 <- paste("mC_count_", strain, "_rep1", sep="")
    cov_r1 <- paste("coverage_", strain, "_rep1", sep="")

    meth_freq_r2 <- paste("methylation_frequency_", strain, "_rep2", sep="")
    m5C_count_r2 <- paste("mC_count_", strain, "_rep2", sep="")
    cov_r2 <- paste("coverage_", strain, "_rep2", sep="")
    
    
    meth_freq_r3 <- paste("methylation_frequency_", strain, "_rep3", sep="")
    m5C_count_r3 <- paste("mC_count_", strain, "_rep3", sep="")
    cov_r3 <- paste("coverage_", strain, "_rep3", sep="")
    
    
    pooled_methylation_frequency_strain <- paste("pooled_methylation_frequency_", strain, sep='')

    #get quantile values for m5C count
    q_TS559_rep1 <- quantile((TS559_cgmap%>%
        filter(coverage_TS559_rep1>=min_cov, mC_count_TS559_rep1>=0))$mC_count_TS559_rep1, .99)[[1]]
    
    q_TS559_rep2 <- quantile((TS559_cgmap%>%
        filter(coverage_TS559_rep2>=min_cov, mC_count_TS559_rep2>=0))$mC_count_TS559_rep2, .99)[[1]]
    
    q_TS559_rep3 <- quantile((TS559_cgmap%>%
        filter(coverage_TS559_rep3>=min_cov, mC_count_TS559_rep3>=0))$mC_count_TS559_rep3, .99)[[1]]

    q_strain_rep1 <- quantile(
        (strain_cgmap %>%
        filter(!!as.name(cov_r1)>=min_cov, !!as.name(m5C_count_r1)>=0) )[[m5C_count_r1]], .99)[[1]]
    
    q_strain_rep2 <- quantile(
        (strain_cgmap %>%
        filter(!!as.name(cov_r2)>=min_cov, !!as.name(m5C_count_r2)>=0) )[[m5C_count_r2]], .99)[[1]]
   
    q_strain_rep3 <- quantile(
        (strain_cgmap %>%
        filter(!!as.name(cov_r3)>=min_cov, !!as.name(m5C_count_r3)>=0) )[[m5C_count_r3]], .99)[[1]] 
    
    #build dataframe
    data <- strain_cgmap %>%
        merge(y = TS559_cgmap, all = TRUE, 
             by= c("chromosome","position","reference_nucleotide","growth_phase","metabolic_condition")) %>%
        dplyr::select(-chromosome,-standard_devition) %>%
    
        # If coverage is NA, it is also 0
        mutate(!!as.name(cov_r1) := replace_na(!!as.name(cov_r1),0)) %>%
        mutate(!!as.name(cov_r2) := replace_na(!!as.name(cov_r2),0)) %>%
        mutate(!!as.name(cov_r3) := replace_na(!!as.name(cov_r3),0)) %>%

        mutate(coverage_TS559_rep1 = replace_na(coverage_TS559_rep1,0)) %>%
        mutate(coverage_TS559_rep2 = replace_na(coverage_TS559_rep2,0)) %>%
        mutate(coverage_TS559_rep3 = replace_na(coverage_TS559_rep3,0)) %>%  
                                                      
        # identify high confidence and reproducible m5C sites
        rowwise() %>%
        mutate(hiconf_TS559 = detect_hiconf_3reps(
            f1=methylation_frequency_TS559_rep1,
            m1=mC_count_TS559_rep1,
            p1=q_TS559_rep1,
            c1=coverage_TS559_rep1,
            f2=methylation_frequency_TS559_rep2,
            m2=mC_count_TS559_rep2,
            p2=q_TS559_rep2,
            c2=coverage_TS559_rep2,
            f3=methylation_frequency_TS559_rep3,
            m3=mC_count_TS559_rep3,
            p3=q_TS559_rep3,
            c3=coverage_TS559_rep3,
            min_cov=min_cov,
            min_freq = minimum_freq) 
              ) %>%
        
        rowwise() %>%
        mutate(hiconf_strain = detect_hiconf_3reps(
                f1=!!as.name(meth_freq_r1),
                m1=!!as.name(m5C_count_r1),
                p1=q_strain_rep1,
                c1=!!as.name(cov_r1),
                f2=!!as.name(meth_freq_r2),
                m2=!!as.name(m5C_count_r2),
                p2=q_strain_rep2,
                c2=!!as.name(cov_r2), 
                f3=!!as.name(meth_freq_r3),
                m3=!!as.name(m5C_count_r3),
                p3=q_strain_rep3,
                c3=!!as.name(cov_r3), 
                min_cov= min_cov,
                min_freq = minimum_freq)
                 
                ) %>%
        # we are only interested in keeping sites were either the parent or deletion
        # strain have a high-confidence m5C site
        filter(hiconf_TS559 == TRUE | hiconf_strain == TRUE) %>%
        filter(!is.na(hiconf_TS559) & !is.na(hiconf_strain)) %>%                                                    
                                                          
        # calculate the mean m5C frequency at each site by pooling replicates (round to 2 decimals)
        # calculate standard deviation of each site as well
        mutate(
                !!as.name(pooled_methylation_frequency_strain) := 
                ( !!as.name(m5C_count_r1) + !!as.name(m5C_count_r2) + !!as.name(m5C_count_r3) )/ 
                ( !!as.name(cov_r1) + !!as.name(cov_r2) + !!as.name(cov_r3) ) 
              ) %>%
    
        
        mutate(!!as.name(pooled_methylation_frequency_strain) := 
               round(!!as.name(pooled_methylation_frequency_strain),2)
              ) %>%
        mutate(strain_sd = sd(c(!!as.name(meth_freq_r1),!!as.name(meth_freq_r2),!!as.name(meth_freq_r3)))
              )%>%
    
        mutate(pooled_methylation_frequency_TS559 =
               (mC_count_TS559_rep1 + mC_count_TS559_rep2 + mC_count_TS559_rep3) /
               (coverage_TS559_rep1+coverage_TS559_rep2+coverage_TS559_rep3)    ) %>%
        mutate(pooled_methylation_frequency_TS559=round(pooled_methylation_frequency_TS559,2)) %>%
        mutate(TS559_sd = sd(c(methylation_frequency_TS559_rep1,methylation_frequency_TS559_rep2,
                              methylation_frequency_TS559_rep3))) %>%
    mutate(CI90_overlap = 
                   rm_overlap(!!as.name(pooled_methylation_frequency_strain), strain_sd,
                    pooled_methylation_frequency_TS559, TS559_sd )) %>%
    
    mutate(difference = get_difference(hiconf_TS559, hiconf_strain,   
                                            !!as.name(pooled_methylation_frequency_strain), 
                                            pooled_methylation_frequency_TS559, CI90_overlap ) ) %>%

        # calculate fold change of methylaton frequency at each site between parent and deletion strain
        mutate(FoldChange = (!!as.name(pooled_methylation_frequency_strain)+.01) / 
                               (pooled_methylation_frequency_TS559+.01),
               log2FC = log2( (!!as.name(pooled_methylation_frequency_strain)+.01) / 
                               (pooled_methylation_frequency_TS559+.01) ) ) %>%
        
        filter(difference == "ABS LOSS" | difference == "REL LOSS" | difference == "ABS GAIN" | 
           difference == "REL GAIN" | difference == "error") %>%
        filter(log2FC >= 1 | log2FC <= -1) %>%
        
        # add p vlaue
        rowwise() %>%
        mutate(pvalue = get_pvalue_3reps(
            s1_mC = !!as.name(m5C_count_r1),
            s1_cov = !!as.name(cov_r1), 
            s2_mC = !!as.name(m5C_count_r2),
            s2_cov = !!as.name(cov_r2),
            s3_mC = !!as.name(m5C_count_r3),
            s3_cov = !!as.name(cov_r3),
            c1_mC = mC_count_TS559_rep1,
            c1_cov = coverage_TS559_rep1,
            c2_mC = mC_count_TS559_rep2,
            c2_cov = coverage_TS559_rep2,
            c3_mC = mC_count_TS559_rep3,
            c3_cov = coverage_TS559_rep3) ) %>%
        
    
        filter(!is.na(pvalue), pvalue <= 0.0)
            
        # adjust p values
        # adj_pvalue <- p.adjust(data$pvalue, method = 'fdr')
        # data <- cbind(data, adj_pvalue)
    
        data <- data %>% merge(y=annotation,by='position', all.x=TRUE) %>%
            dplyr::rename(TS559_position = position) %>%
           dplyr::select(-reproducible, -highly_reproducible, -reference_nucleotide) %>%
           arrange(log2FC) %>%
        dplyr::select(TS559_position, KOD1_position, strand, everything())
    
    return(data)
}

enumerate_hiconf_3reps <- function(strain_cgmap, strain, min_cov=47){
    
    meth_freq_r1 <- paste("methylation_frequency_", strain, "_rep1", sep="")
    m5C_count_r1 <- paste("mC_count_", strain, "_rep1", sep="")
    cov_r1 <- paste("coverage_", strain, "_rep1", sep="")

    meth_freq_r2 <- paste("methylation_frequency_", strain, "_rep2", sep="")
    m5C_count_r2 <- paste("mC_count_", strain, "_rep2", sep="")
    cov_r2 <- paste("coverage_", strain, "_rep2", sep="")
    
    q_strain_rep1 <- quantile(
        (strain_cgmap %>%
        filter(!!as.name(cov_r1)>=min_cov, !!as.name(m5C_count_r1)>=0) )[[m5C_count_r1]], .99)[[1]]
    
    q_strain_rep2 <- quantile(
        (strain_cgmap %>%
        filter(!!as.name(cov_r2)>=min_cov, !!as.name(m5C_count_r2)>=0) )[[m5C_count_r2]], .99)[[1]]
    
    
    data <- strain_cgmap %>%
        dplyr::select(-chromosome, -reference_nucleotide,-growth_phase) %>%
    
        mutate(!!as.name(cov_r1) := replace_na(!!as.name(cov_r1),0)) %>%
        mutate(!!as.name(cov_r2) := replace_na(!!as.name(cov_r2),0)) %>%   
        
        rowwise() %>%
        mutate(hiconf_strain = detect_hiconf_2reps(
                f1=!!as.name(meth_freq_r1),
                m1=!!as.name(m5C_count_r1),
                p1=q_strain_rep1,
                c1=!!as.name(cov_r1),
                f2=!!as.name(meth_freq_r2),
                m2=!!as.name(m5C_count_r2),
                p2=q_strain_rep2,
                c2=!!as.name(cov_r2), 
                min_cov=min_cov)
                 
                ) %>%
        dplyr::rename(TS559_position = position) %>%
        
        mutate( 'm5C_cov/total_cov_rep1' := 
            paste(!!as.name(paste('mC_count_',strain,'_rep1',sep='')), 
                  '/', 
                  !!as.name(paste('coverage_',strain,'_rep1',sep='')) 
                 ) 
              ) %>%
        
        mutate( 'm5C_cov/total_cov_rep2' := 
            paste(!!as.name(paste('mC_count_',strain,'_rep2',sep='')), 
                  '/', 
                  !!as.name(paste('coverage_',strain,'_rep2',sep='')) 
                 ) 
              ) %>%
        dplyr::select(TS559_position, 
                  !!as.name(meth_freq_r1), 
                  'm5C_cov/total_cov_rep1',
                  !!as.name(meth_freq_r2), 
                  'm5C_cov/total_cov_rep2',
                     hiconf_strain) %>%

    dplyr::rename(!!paste(strain,'_hiconf',sep=''):= hiconf_strain )  %>%
    dplyr::rename(!!paste('m5C_cov/total_cov_',strain,'_rep1',sep=''):= 'm5C_cov/total_cov_rep1' )  %>%
    dplyr::rename(!!paste('m5C_cov/total_cov_',strain,'_rep2',sep=''):= 'm5C_cov/total_cov_rep2' ) %>%
    dplyr::rename(!!paste('freq_',strain,'_rep1',sep=''):= !!as.name(paste('methylation_frequency_',strain,'_rep1',sep='') ) )%>%
    dplyr::rename(!!paste('freq_',strain,'_rep2',sep=''):= !!as.name(paste('methylation_frequency_',strain,'_rep2',sep='') ) )  
    
    return(data)
}

In [5]:
# TS559 --  control/parent strain

TS559statS_CGmap_rep1 <-load_cgmap("../cgmaps/TS559statS_totalRNA_March2016.CGmap",
    strain="TS559", growth_phase='stationary', metabolic_condition="S", replicate="rep1") %>%
    dplyr::rename(reference_nucleotide=base)

TS559statS_CGmap_rep2 <-load_cgmap("../cgmaps/TS559statS_totalRNA_Dec2016.CGmap", 
    strain="TS559", growth_phase='stationary', metabolic_condition="S", replicate="rep2")%>%
    dplyr::rename(reference_nucleotide=base)

TS559statS_CGmap_rep3 <-load_cgmap("../cgmaps/TS559statS_totalRNA_Oct2020.CGmap",
    strain="TS559", growth_phase='stationary', metabolic_condition="S", replicate="rep3")%>%
    dplyr::rename(reference_nucleotide=base)
   
    
TS559statS_rep1_df <- processCGmap(TS559statS_CGmap_rep1)
TS559statS_rep2_df <- processCGmap(TS559statS_CGmap_rep2)
TS559statS_rep3_df <- processCGmap(TS559statS_CGmap_rep3)


m5C_count_e1 = 6
m5C_count_e2 = 18
m5C_count_e3 = 7
m5C_count_s1 = 5
m5C_count_s2 = 17
m5C_count_s3 = 5

TS559statS_all <- merge(x=TS559statS_rep1_df, y= TS559statS_rep2_df, 
                   by=c("chromosome","position", 'reference_nucleotide', 'growth_phase','metabolic_condition'), 
                   all=TRUE) %>%
            merge(y=TS559statS_rep3_df, 
                   by=c("chromosome","position", 'reference_nucleotide', 'growth_phase','metabolic_condition'), 
                   all=TRUE) %>%
            mutate(coverage_TS559_rep1 = replace_na(coverage_TS559_rep1,0)) %>%
            mutate(coverage_TS559_rep2 = replace_na(coverage_TS559_rep2,0)) %>%
            mutate(coverage_TS559_rep3 = replace_na(coverage_TS559_rep3,0)) %>%
            rowwise()%>%
            mutate(reproducible = detect_hiconf_3reps(
                f1=methylation_frequency_TS559_rep1,
                m1=mC_count_TS559_rep1,
                p1=m5C_count_e1,
                c1=coverage_TS559_rep1,
                f2=methylation_frequency_TS559_rep2,
                m2=mC_count_TS559_rep2,
                p2=m5C_count_e2,
                c2=coverage_TS559_rep2,
                f3=methylation_frequency_TS559_rep3,
                m3=mC_count_TS559_rep3,
                p3=m5C_count_e3,
                c3=coverage_TS559_rep3, 
                min_cov=47))

head(TS559statS_all)

TS559statS_cat <-TS559statS_all %>%
    dplyr::select(-chromosome, -metabolic_condition) %>%
    dplyr::rename(TS559_position = position, TS559_hiconf = reproducible) %>%
    dplyr::rename(freq_TS559_rep1 = methylation_frequency_TS559_rep1, freq_TS559_rep2 = methylation_frequency_TS559_rep2, 
                  freq_TS559_rep3=methylation_frequency_TS559_rep3) %>%
    mutate('m5C_cov/total_cov_TS559_rep1' = paste(mC_count_TS559_rep1, '/', coverage_TS559_rep1) ) %>%
    mutate('m5C_cov/total_cov_TS559_rep2' = paste(mC_count_TS559_rep2, '/', coverage_TS559_rep2) ) %>%
    mutate('m5C_cov/total_cov_TS559_rep3' = paste(mC_count_TS559_rep3, '/', coverage_TS559_rep3) ) %>%
    dplyr::select(-mC_count_TS559_rep1,-coverage_TS559_rep1,-mC_count_TS559_rep2, 
                  -coverage_TS559_rep2,-mC_count_TS559_rep3, -coverage_TS559_rep3) %>%
    dplyr::select("TS559_position","freq_TS559_rep1", 
          "m5C_cov/total_cov_TS559_rep1","freq_TS559_rep2","m5C_cov/total_cov_TS559_rep2",
           "freq_TS559_rep3","m5C_cov/total_cov_TS559_rep3", 'TS559_hiconf'
          )        

Sys.sleep(10)


chromosome,position,reference_nucleotide,growth_phase,metabolic_condition,methylation_frequency_TS559_rep1,mC_count_TS559_rep1,coverage_TS559_rep1,methylation_frequency_TS559_rep2,mC_count_TS559_rep2,coverage_TS559_rep2,methylation_frequency_TS559_rep3,mC_count_TS559_rep3,coverage_TS559_rep3,reproducible
<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<lgl>
TS559_Genomic_Sequence.seq,6,C,stationary,S,0,0,12,0,0,78,0,0,158,FALSE
TS559_Genomic_Sequence.seq,7,C,stationary,S,0,0,13,0,0,78,0,0,164,FALSE
TS559_Genomic_Sequence.seq,9,C,stationary,S,0,0,13,0,0,82,0,0,170,FALSE
TS559_Genomic_Sequence.seq,12,C,stationary,S,0,0,13,0,0,83,0,0,171,FALSE
TS559_Genomic_Sequence.seq,14,C,stationary,S,0,0,13,0,0,83,0,0,172,FALSE
TS559_Genomic_Sequence.seq,18,C,stationary,S,0,0,14,0,0,84,0,0,173,FALSE


In [6]:
# ###strain TK2045

# defaultW <- getOption("warn")
# options(warn = -1)


# #load CGmaps into session
# TK2045statS_CGmap_rep1 <-read.delim("../cgmaps/TK2045statS_totalRNA_Dec2016.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
#     mutate(strain="TK2045", growth_phase='stationary', metabolic_condition="S", replicate="rep1")

# TK2045statS_CGmap_rep2 <-read.delim("../cgmaps/TK2045statS_totalRNA_Oct2020.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
#     mutate(strain="TK2045", growth_phase='stationary', metabolic_condition="S", replicate="rep2")

# # process raw CGmaps
# TK2045statS_rep1_df <- processCGmap(TK2045statS_CGmap_rep1)
# TK2045statS_rep2_df <- processCGmap(TK2045statS_CGmap_rep2)

# #merge reps
# TK2045statS <- merge_del_reps(x=TK2045statS_rep1_df, y=TK2045statS_rep2_df, strain='TK2045')

# # get quantiles
# "quantiles:"
# get_quantiles(strain_cgmap = TK2045statS, strain="TK2045", min_cov = 47)

# #compare methylation frequencies & annotate
# TK2045statS_anal <- CompareCGmap(strain_cgmap = TK2045statS, 
#                                 TS559_cgmap = TS559statS, 
#                                 strain="TK2045", 
#                                 annotation = annotation,
#                                 min_cov = 47)

# write.table(TK2045statS_anal, file = "../processed_cgmaps/TK2045statS_annotated", sep = "\t", row.names = F)

# #enumerate gains & losses
# "losses & gains:"
# nrow( TK2045statS_anal %>% filter(log2FC < 0) )
# nrow( TK2045statS_anal %>% filter(log2FC > 0) )

# "absolute losses & gains:"
# nrow( TK2045statS_anal %>% filter(difference == "ABS LOSS") )
# nrow( TK2045statS_anal %>% filter(difference == "ABS GAIN") )

# head(TK2045statS_anal)

# TK2045statS_cat <- enumerate_hiconf_2reps(strain_cgmap = TK2045statS, strain="TK2045",min_cov = 47)

# Sys.sleep(10)


In [7]:
###strain TK0224

defaultW <- getOption("warn")
options(warn = -1)


#load CGmaps into session
TK0224statS_CGmap_rep1 <-read.delim("../cgmaps/TK0224statS_totalRNA_Dec2016.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK0224", growth_phase='stationary', metabolic_condition="S", replicate="rep1")

TK0224statS_CGmap_rep2 <-read.delim("../cgmaps/TK0224statS_totalRNA_Oct2020.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK0224", growth_phase='stationary', metabolic_condition="S", replicate="rep2")

# process raw CGmaps
TK0224statS_rep1_df <- processCGmap(TK0224statS_CGmap_rep1)
TK0224statS_rep2_df <- processCGmap(TK0224statS_CGmap_rep2)

#merge reps
TK0224statS <- merge_del_reps(x=TK0224statS_rep1_df, y=TK0224statS_rep2_df, strain='TK0224')

# get quantiles
"quantiles:"
get_quantiles(strain_cgmap = TK0224statS, strain="TK0224", min_cov = 47)

#compare methylation frequencies & annotate
TK0224statS_anal <- CompareCGmap(strain_cgmap = TK0224statS, 
                                TS559_cgmap = TS559statS, 
                                strain="TK0224", 
                                annotation = annotation,
                                min_cov = 47)

write.table(TK0224statS_anal, file = "../processed_cgmaps/TK0224statS_annotated", sep = "\t", row.names = F)

#enumerate gains & losses
"losses & gains:"
nrow( TK0224statS_anal %>% filter(log2FC < 0) )
nrow( TK0224statS_anal %>% filter(log2FC > 0) )

"absolute losses & gains:"
nrow( TK0224statS_anal %>% filter(difference == "ABS LOSS") )
nrow( TK0224statS_anal %>% filter(difference == "ABS GAIN") )


head(TK0224statS_anal)

# TK0224statS_cat <- enumerate_hiconf_2reps(strain_cgmap = TK0224statS, strain="TK0224",min_cov = 47)

Sys.sleep(10)

[1] "quantiles:"

[1] 5 7

In [8]:
###strain TK0234

defaultW <- getOption("warn")
options(warn = -1)


#load CGmaps into session
TK0234statS_CGmap_rep1 <-read.delim("../cgmaps/TK0234statS_totalRNA_Dec2016.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK0234", growth_phase='stationary', metabolic_condition="S", replicate="rep1")

TK0234statS_CGmap_rep2 <-read.delim("../cgmaps/TK0234statS_totalRNA_Oct2020.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK0234", growth_phase='stationary', metabolic_condition="S", replicate="rep2")

# process raw CGmaps
TK0234statS_rep1_df <- processCGmap(TK0234statS_CGmap_rep1)
TK0234statS_rep2_df <- processCGmap(TK0234statS_CGmap_rep2)

#merge reps
TK0234statS <- merge_del_reps(x=TK0234statS_rep1_df, y=TK0234statS_rep2_df, strain='TK0234')

# get quantiles
"quantiles:"
get_quantiles(strain_cgmap = TK0234statS, strain="TK0234", min_cov = 47)

#compare methylation frequencies & annotate
TK0234statS_anal <- CompareCGmap(strain_cgmap = TK0234statS, 
                                TS559_cgmap = TS559statS, 
                                strain="TK0234", 
                                annotation = annotation,
                                min_cov = 47)

write.table(TK0234statS_anal, file = "../processed_cgmaps/TK0234statS_annotated", sep = "\t", row.names = F)

#enumerate gains & losses
"losses & gains:"
nrow( TK0234statS_anal %>% filter(log2FC < 0) )
nrow( TK0234statS_anal %>% filter(log2FC > 0) )

"absolute losses & gains:"
nrow( TK0234statS_anal %>% filter(difference == "ABS LOSS") )
nrow( TK0234statS_anal %>% filter(difference == "ABS GAIN") )

head(TK0234statS_anal)

# TK0234statS_cat <- enumerate_hiconf_2reps(strain_cgmap = TK0234statS, strain="TK0234",min_cov = 47)

Sys.sleep(10)

[1] "quantiles:"

[1] 5 7

In [7]:
###strain TK0360

defaultW <- getOption("warn")
options(warn = -1)


#load CGmaps into session
TK0360statS_CGmap_rep1 <-read.delim("../cgmaps/TK0360_statS_totalRNA_rep1.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK0360", growth_phase='stationary', metabolic_condition="S", replicate="rep1")

TK0360statS_CGmap_rep2 <-read.delim("../cgmaps/TK0360_statS_totalRNA_rep2.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK0360", growth_phase='stationary', metabolic_condition="S", replicate="rep2")

# process raw CGmaps
TK0360statS_rep1_df <- processCGmap(TK0360statS_CGmap_rep1)
TK0360statS_rep2_df <- processCGmap(TK0360statS_CGmap_rep2)

#merge reps
TK0360statS <- merge_del_reps(x=TK0360statS_rep1_df, y=TK0360statS_rep2_df, strain='TK0360')

# get quantiles
"quantiles:"
get_quantiles(strain_cgmap = TK0360statS, strain="TK0360", min_cov = 47)

#compare methylation frequencies & annotate
TK0360statS_anal <- CompareCGmap(strain_cgmap = TK0360statS, 
                                TS559_cgmap = TS559statS, 
                                strain="TK0360", 
                                annotation = annotation,
                                min_cov = 47)

write.table(TK0360statS_anal, file = "../processed_cgmaps/TK0360statS_annotated", sep = "\t", row.names = F)

#enumerate gains & losses
"losses & gains:"
nrow( TK0360statS_anal %>% filter(log2FC < 0) )
nrow( TK0360statS_anal %>% filter(log2FC > 0) )

"absolute losses & gains:"
nrow( TK0360statS_anal %>% filter(difference == "ABS LOSS") )
nrow( TK0360statS_anal %>% filter(difference == "ABS GAIN") )

head(TK0360statS_anal)

# TK0360statS_cat <- enumerate_hiconf_2reps(strain_cgmap = TK0360statS, strain="TK0360",min_cov = 47)

Sys.sleep(10)

[1] "quantiles:"

[1] 9 9

[1] "losses & gains:"

[1] 2

[1] 34

[1] "absolute losses & gains:"

[1] 1

[1] 12

,TS559_position,KOD1_position,strand,growth_phase,metabolic_condition,methylation_frequency_TK0360_rep1,mC_count_TK0360_rep1,coverage_TK0360_rep1,methylation_frequency_TK0360_rep2,mC_count_TK0360_rep2,⋯,amino_acid_sequence,amino_acid_ID,local_41bp_predicted_fold,m5C_position_fold,MFE,associated_TSS_id,TSS_direction,TSS_description,total_annotations,alternate_annotations
,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>,<int>,<dbl>,<int>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>
1,1772255,1773246,-,stationary,S,0.05,120,2355,0.06,157,⋯,CCG,P,|..|.......|||||..|||.|||..||..|.........,base_paired,-68.5,.,.,.,1,NA
2,2019350,2020341,-,stationary,S,0.04,57,1371,0.05,68,⋯,GAC,D,||||||||...|||..||..................||..|,single_stranded,-22.79,.,.,.,1,NA
3,685151,686142,-,stationary,S,0.06,61,950,0.05,37,⋯,.,.,.,NA,.,.,.,.,1,NA
4,1344043,1345034,-,stationary,S,0.06,88,1373,0.06,89,⋯,GAC,D,................................||..|..||,single_stranded,-25.73,.,.,.,1,NA
5,1480381,1481372,-,stationary,S,0.06,80,1316,0.05,68,⋯,GCG,A,||||...|||||||||...|||||.........|||..|||,base_paired,-36.73,.,.,.,1,NA
6,1492125,1493116,-,stationary,S,0.07,57,876,0.06,61,⋯,CCG,P,|....|||||||||..|||||.................|..,base_paired,-26.58,.,.,.,1,NA


In [10]:
###strain TK0704

defaultW <- getOption("warn")
options(warn = -1)


#load CGmaps into session
TK0704statS_CGmap_rep1 <-read.delim("../cgmaps/TK0704statS_totalRNA_Dec2016.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK0704", growth_phase='stationary', metabolic_condition="S", replicate="rep1")

TK0704statS_CGmap_rep2 <-read.delim("../cgmaps/TK0704statS_totalRNA_Oct2020.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK0704", growth_phase='stationary', metabolic_condition="S", replicate="rep2")

# process raw CGmaps
TK0704statS_rep1_df <- processCGmap(TK0704statS_CGmap_rep1)
TK0704statS_rep2_df <- processCGmap(TK0704statS_CGmap_rep2)

#merge reps
TK0704statS <- merge_del_reps(x=TK0704statS_rep1_df, y=TK0704statS_rep2_df, strain='TK0704')

# get quantiles
"quantiles:"
get_quantiles(strain_cgmap = TK0704statS, strain="TK0704", min_cov = 47)

#compare methylation frequencies & annotate
TK0704statS_anal <- CompareCGmap(strain_cgmap = TK0704statS, 
                                TS559_cgmap = TS559statS, 
                                strain="TK0704", 
                                annotation = annotation,
                                min_cov = 47)

write.table(TK0704statS_anal, file = "../processed_cgmaps/TK0704statS_annotated", sep = "\t", row.names = F)

#enumerate gains & losses
"losses & gains:"
nrow( TK0704statS_anal %>% filter(log2FC < 0) )
nrow( TK0704statS_anal %>% filter(log2FC > 0) )

"absolute losses & gains:"
nrow( TK0704statS_anal %>% filter(difference == "ABS LOSS") )
nrow( TK0704statS_anal %>% filter(difference == "ABS GAIN") )


head(TK0704statS_anal)

# TK0704statS_cat <- enumerate_hiconf_2reps(strain_cgmap = TK0704statS, strain="TK0704",min_cov = 47)

Sys.sleep(10)

[1] "quantiles:"

[1] 14  6

In [11]:
###strain TK0729

defaultW <- getOption("warn")
options(warn = -1)


#load CGmaps into session
TK0729statS_CGmap_rep1 <-read.delim("../cgmaps/TK0729statS_totalRNA_Dec2016.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK0729", growth_phase='stationary', metabolic_condition="S", replicate="rep1")

TK0729statS_CGmap_rep2 <-read.delim("../cgmaps/TK0729statS_totalRNA_Oct2020.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK0729", growth_phase='stationary', metabolic_condition="S", replicate="rep2")

# process raw CGmaps
TK0729statS_rep1_df <- processCGmap(TK0729statS_CGmap_rep1)
TK0729statS_rep2_df <- processCGmap(TK0729statS_CGmap_rep2)

#merge reps
TK0729statS <- merge_del_reps(x=TK0729statS_rep1_df, y=TK0729statS_rep2_df, strain='TK0729')

# get quantiles
"quantiles:"
get_quantiles(strain_cgmap = TK0729statS, strain="TK0729", min_cov = 47)

#compare methylation frequencies & annotate
TK0729statS_anal <- CompareCGmap(strain_cgmap = TK0729statS, 
                                TS559_cgmap = TS559statS, 
                                strain="TK0729", 
                                annotation = annotation,
                                min_cov = 47)

write.table(TK0729statS_anal, file = "../processed_cgmaps/TK0729statS_annotated", sep = "\t", row.names = F)

#enumerate gains & losses
"losses & gains:"
nrow( TK0729statS_anal %>% filter(log2FC < 0) )
nrow( TK0729statS_anal %>% filter(log2FC > 0) )

"absolute losses & gains:"
nrow( TK0729statS_anal %>% filter(difference == "ABS LOSS") )
nrow( TK0729statS_anal %>% filter(difference == "ABS GAIN") )

head(TK0729statS_anal)

# TK0729statS_cat <- enumerate_hiconf_2reps(strain_cgmap = TK0729statS, strain="TK0729",min_cov = 47)

Sys.sleep(10)

[1] "quantiles:"

[1] 5 6

In [8]:
###strain TK0872

defaultW <- getOption("warn")
options(warn = -1)


#load CGmaps into session
TK0872statS_CGmap_rep1 <-read.delim("../cgmaps/TK0872statS_totalRNA_Dec2016.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK0872", growth_phase='stationary', metabolic_condition="S", replicate="rep1")

TK0872statS_CGmap_rep2 <-read.delim("../cgmaps/TK0872statS_totalRNA_Oct2020.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK0872", growth_phase='stationary', metabolic_condition="S", replicate="rep2")

# process raw CGmaps
TK0872statS_rep1_df <- processCGmap(TK0872statS_CGmap_rep1)
TK0872statS_rep2_df <- processCGmap(TK0872statS_CGmap_rep2)

#merge reps
TK0872statS <- merge_del_reps(x=TK0872statS_rep1_df, y=TK0872statS_rep2_df, strain='TK0872')

# get quantiles
"quantiles:"
get_quantiles(strain_cgmap = TK0872statS, strain="TK0872", min_cov = 47)

#compare methylation frequencies & annotate
TK0872statS_anal <- CompareCGmap(strain_cgmap = TK0872statS, 
                                TS559_cgmap = TS559statS, 
                                strain="TK0872", 
                                annotation = annotation,
                                min_cov = 47)

write.table(TK0872statS_anal, file = "../processed_cgmaps/TK0872statS_annotated", sep = "\t", row.names = F)

#enumerate gains & losses
"losses & gains:"
nrow( TK0872statS_anal %>% filter(log2FC < 0) )
nrow( TK0872statS_anal %>% filter(log2FC > 0) )

"absolute losses & gains:"
nrow( TK0872statS_anal %>% filter(difference == "ABS LOSS") )
nrow( TK0872statS_anal %>% filter(difference == "ABS GAIN") )

head(TK0872statS_anal)

# TK0872statS_cat <- enumerate_hiconf_2reps(strain_cgmap = TK0872statS, strain="TK0872",min_cov = 47)

Sys.sleep(10)

[1] "quantiles:"

[1] 11  5

[1] "losses & gains:"

[1] 4

[1] 0

[1] "absolute losses & gains:"

[1] 4

[1] 0

,TS559_position,KOD1_position,strand,growth_phase,metabolic_condition,methylation_frequency_TK0872_rep1,mC_count_TK0872_rep1,coverage_TK0872_rep1,methylation_frequency_TK0872_rep2,mC_count_TK0872_rep2,⋯,amino_acid_sequence,amino_acid_ID,local_41bp_predicted_fold,m5C_position_fold,MFE,associated_TSS_id,TSS_direction,TSS_description,total_annotations,alternate_annotations
,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>,<int>,<dbl>,<int>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>
1,2026195,2027186,+,stationary,S,0,0,423,0.03,10,⋯,.,.,|..||||||||........|||||||||||||.||||||||,base_paired,-440.69,.,.,.,1,NA
2,611853,612844,+,stationary,S,0,1,946,0.01,17,⋯,CUC,L,|....|||||||||.....||||....|||||..|..||||,base_paired,-86.58,.,.,.,1,NA
3,793629,794620,+,stationary,S,0,0,178,0.02,4,⋯,CUC,L,||||||||..........||||..||..|........||||,base_paired,-122.17,.,.,.,1,NA
4,1868187,1869178,+,stationary,S,0,0,1819,0.00,10,⋯,CUC,L,.|||..|......|||||||..||...||||..........,single_stranded,-31.45,.,.,.,1,NA


In [13]:
###strain TK1273

defaultW <- getOption("warn")
options(warn = -1)


#load CGmaps into session
TK1273statS_CGmap_rep1 <-read.delim("../cgmaps/TK1273statS_totalRNA_Dec2016.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK1273", growth_phase='stationary', metabolic_condition="S", replicate="rep1")

TK1273statS_CGmap_rep2 <-read.delim("../cgmaps/TK1273statS_totalRNA_Oct2020.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK1273", growth_phase='stationary', metabolic_condition="S", replicate="rep2")

# process raw CGmaps
TK1273statS_rep1_df <- processCGmap(TK1273statS_CGmap_rep1)
TK1273statS_rep2_df <- processCGmap(TK1273statS_CGmap_rep2)

#merge reps
TK1273statS <- merge_del_reps(x=TK1273statS_rep1_df, y=TK1273statS_rep2_df, strain='TK1273')

# get quantiles
"quantiles:"
get_quantiles(strain_cgmap = TK1273statS, strain="TK1273", min_cov = 47)

#compare methylation frequencies & annotate
TK1273statS_anal <- CompareCGmap(strain_cgmap = TK1273statS, 
                                TS559_cgmap = TS559statS, 
                                strain="TK1273", 
                                annotation = annotation,
                                min_cov = 47)

write.table(TK1273statS_anal, file = "../processed_cgmaps/TK1273statS_annotated", sep = "\t", row.names = F)

#enumerate gains & losses
"losses & gains:"
nrow( TK1273statS_anal %>% filter(log2FC < 0) )
nrow( TK1273statS_anal %>% filter(log2FC > 0) )

"absolute losses & gains:"
nrow( TK1273statS_anal %>% filter(difference == "ABS LOSS") )
nrow( TK1273statS_anal %>% filter(difference == "ABS GAIN") )

head(TK1273statS_anal)

# TK1273statS_cat <- enumerate_hiconf_2reps(strain_cgmap = TK1273statS, strain="TK1273",min_cov = 47)

Sys.sleep(10)

[1] "quantiles:"

[1] 16  5

In [14]:
###strain TK1784

defaultW <- getOption("warn")
options(warn = -1)


#load CGmaps into session
TK1784statS_CGmap_rep1 <-read.delim("../cgmaps/TK1784_statS_totalRNA_rep1.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK1784", growth_phase='stationary', metabolic_condition="S", replicate="rep1")

TK1784statS_CGmap_rep2 <-read.delim("../cgmaps/TK1784_statS_totalRNA_rep2.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK1784", growth_phase='stationary', metabolic_condition="S", replicate="rep2")

# process raw CGmaps
TK1784statS_rep1_df <- processCGmap(TK1784statS_CGmap_rep1)
TK1784statS_rep2_df <- processCGmap(TK1784statS_CGmap_rep2)

#merge reps
TK1784statS <- merge_del_reps(x=TK1784statS_rep1_df, y=TK1784statS_rep2_df, strain='TK1784')

# get quantiles
"quantiles:"
get_quantiles(strain_cgmap = TK1784statS, strain="TK1784", min_cov = 47)

#compare methylation frequencies & annotate
TK1784statS_anal <- CompareCGmap(strain_cgmap = TK1784statS, 
                                TS559_cgmap = TS559statS, 
                                strain="TK1784", 
                                annotation = annotation,
                                min_cov = 47)

write.table(TK1784statS_anal, file = "../processed_cgmaps/TK1784statS_annotated", sep = "\t", row.names = F)

#enumerate gains & losses
"losses & gains:"
nrow( TK1784statS_anal %>% filter(log2FC < 0) )
nrow( TK1784statS_anal %>% filter(log2FC > 0) )

"absolute losses & gains:"
nrow( TK1784statS_anal %>% filter(difference == "ABS LOSS") )
nrow( TK1784statS_anal %>% filter(difference == "ABS GAIN") )

head(TK1784statS_anal)

# TK1784statS_cat <- enumerate_hiconf_2reps(strain_cgmap = TK1784statS, strain="TK1784",min_cov = 47)

Sys.sleep(10)

[1] "quantiles:"

[1] 10 11

In [15]:
###strain TK1917

defaultW <- getOption("warn")
options(warn = -1)


#load CGmaps into session
TK1917statS_CGmap_rep1 <-read.delim("../cgmaps/TK1917statS_totalRNA_Dec2016.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK1917", growth_phase='stationary', metabolic_condition="S", replicate="rep1")

TK1917statS_CGmap_rep2 <-read.delim("../cgmaps/TK1917statS_totalRNA_Oct2020.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK1917", growth_phase='stationary', metabolic_condition="S", replicate="rep2")

# process raw CGmaps
TK1917statS_rep1_df <- processCGmap(TK1917statS_CGmap_rep1)
TK1917statS_rep2_df <- processCGmap(TK1917statS_CGmap_rep2)

#merge reps
TK1917statS <- merge_del_reps(x=TK1917statS_rep1_df, y=TK1917statS_rep2_df, strain='TK1917')

# get quantiles
"quantiles:"
get_quantiles(strain_cgmap = TK1917statS, strain="TK1917", min_cov = 47)

#compare methylation frequencies & annotate
TK1917statS_anal <- CompareCGmap(strain_cgmap = TK1917statS, 
                                TS559_cgmap = TS559statS, 
                                strain="TK1917", 
                                annotation = annotation,
                                min_cov = 47)

write.table(TK1917statS_anal, file = "../processed_cgmaps/TK1917statS_annotated", sep = "\t", row.names = F)

#enumerate gains & losses
"losses & gains:"
nrow( TK1917statS_anal %>% filter(log2FC < 0) )
nrow( TK1917statS_anal %>% filter(log2FC > 0) )
"absolute losses & gains:"
nrow( TK1917statS_anal %>% filter(difference == "ABS LOSS") )
nrow( TK1917statS_anal %>% filter(difference == "ABS GAIN") )

head(TK1917statS_anal)

# TK1917statS_cat <- enumerate_hiconf_2reps(strain_cgmap = TK1917statS, strain="TK1917",min_cov = 47)

Sys.sleep(10)

[1] "quantiles:"

[1] 5 5

In [9]:
###strain TK1935

defaultW <- getOption("warn")
options(warn = -1)


#load CGmaps into session
TK1935statS_CGmap_rep1 <-read.delim("../cgmaps/TK1935statS_totalRNA_Dec2016.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK1935", growth_phase='stationary', metabolic_condition="S", replicate="rep1")

TK1935statS_CGmap_rep2 <-read.delim("../cgmaps/TK1935statS_totalRNA_Oct2020.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK1935", growth_phase='stationary', metabolic_condition="S", replicate="rep2")

# process raw CGmaps
TK1935statS_rep1_df <- processCGmap(TK1935statS_CGmap_rep1)
TK1935statS_rep2_df <- processCGmap(TK1935statS_CGmap_rep2)

#merge reps
TK1935statS <- merge_del_reps(x=TK1935statS_rep1_df, y=TK1935statS_rep2_df, strain='TK1935')

# get quantiles
"quantiles:"
get_quantiles(strain_cgmap = TK1935statS, strain="TK1935", min_cov = 47)

#compare methylation frequencies & annotate
TK1935statS_anal <- CompareCGmap(strain_cgmap = TK1935statS, 
                                TS559_cgmap = TS559statS, 
                                strain="TK1935", 
                                annotation = annotation,
                                min_cov = 47)

write.table(TK1935statS_anal, file = "../processed_cgmaps/TK1935statS_annotated", sep = "\t", row.names = F)

#enumerate gains & losses
"losses & gains:"
nrow( TK1935statS_anal %>% filter(log2FC < 0) )
nrow( TK1935statS_anal %>% filter(log2FC > 0) )

"absolute losses & gains:"
nrow( TK1935statS_anal %>% filter(difference == "ABS LOSS") )
nrow( TK1935statS_anal %>% filter(difference == "ABS GAIN") )

head(TK1935statS_anal)

# TK1935statS_cat <- enumerate_hiconf_2reps(strain_cgmap = TK1935statS, strain="TK1935",min_cov = 47)

Sys.sleep(10)

[1] "quantiles:"

[1] 5 6

[1] "losses & gains:"

[1] 11

[1] 1

[1] "absolute losses & gains:"

[1] 11

[1] 1

,TS559_position,KOD1_position,strand,growth_phase,metabolic_condition,methylation_frequency_TK1935_rep1,mC_count_TK1935_rep1,coverage_TK1935_rep1,methylation_frequency_TK1935_rep2,mC_count_TK1935_rep2,⋯,amino_acid_sequence,amino_acid_ID,local_41bp_predicted_fold,m5C_position_fold,MFE,associated_TSS_id,TSS_direction,TSS_description,total_annotations,alternate_annotations
,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>,<int>,<dbl>,<int>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>
1,1772255,1773246,-,stationary,S,0.06,15,249,0.06,67,⋯,CCG,P,|..|.......|||||..|||.|||..||..|.........,base_paired,-68.5,.,.,.,1,NA
2,2022329,2023320,+,stationary,S,0.00,3,4533,0.00,4,⋯,.,.,||.........|||||...||||||....|||.....|||.,base_paired,-224.51,.,.,.,1,NA
3,2025660,2026651,+,stationary,S,0.00,1,3023,0.00,0,⋯,.,.,........||||||.....||||||................,base_paired,-440.69,.,.,.,1,NA
4,2026031,2027022,+,stationary,S,0.00,4,7803,0.00,3,⋯,.,.,...||||...|||||....||||||||...|||||||....,base_paired,-440.69,.,.,.,1,NA
5,1010753,1011744,-,stationary,S,0.00,0,34,0.00,0,⋯,CUG,L,.....|....|.........|..||||......|...|||.,base_paired,-62.39,.,.,.,1,NA
6,2080544,2082176,+,stationary,S,0.00,0,121,0.00,0,⋯,UGC,C,...||||.....|||||....|||........|..|..|||,single_stranded,-55.88,.,.,.,1,NA


In [10]:
###strain TK2122

defaultW <- getOption("warn")
options(warn = -1)


#load CGmaps into session
TK2122statS_CGmap_rep1 <-read.delim("../cgmaps/TK2122statS_totalRNA_Dec2016.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK2122", growth_phase='stationary', metabolic_condition="S", replicate="rep1")

TK2122statS_CGmap_rep2 <-read.delim("../cgmaps/TK2122statS_totalRNA_Oct2020.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK2122", growth_phase='stationary', metabolic_condition="S", replicate="rep2")

# process raw CGmaps
TK2122statS_rep1_df <- processCGmap(TK2122statS_CGmap_rep1)
TK2122statS_rep2_df <- processCGmap(TK2122statS_CGmap_rep2)

#merge reps
TK2122statS <- merge_del_reps(x=TK2122statS_rep1_df, y=TK2122statS_rep2_df, strain='TK2122')

# get quantiles
"quantiles:"
get_quantiles(strain_cgmap = TK2122statS, strain="TK2122", min_cov = 47)

#compare methylation frequencies & annotate
TK2122statS_anal <- CompareCGmap(strain_cgmap = TK2122statS, 
                                TS559_cgmap = TS559statS, 
                                strain="TK2122", 
                                annotation = annotation,
                                min_cov = 47)

write.table(TK2122statS_anal, file = "../processed_cgmaps/TK2122statS_annotated", sep = "\t", row.names = F)

#enumerate gains & losses
"losses & gains:"
nrow( TK2122statS_anal %>% filter(log2FC < 0) )
nrow( TK2122statS_anal %>% filter(log2FC > 0) )

"absolute losses & gains:"
nrow( TK2122statS_anal %>% filter(difference == "ABS LOSS") )
nrow( TK2122statS_anal %>% filter(difference == "ABS GAIN") )

head(TK2122statS_anal)

# TK2122statS_cat <- enumerate_hiconf_2reps(strain_cgmap = TK2122statS, strain="TK2122",min_cov = 47)

Sys.sleep(10)

[1] "quantiles:"

[1] 16  6

[1] "losses & gains:"

[1] 4

[1] 1

[1] "absolute losses & gains:"

[1] 4

[1] 0

,TS559_position,KOD1_position,strand,growth_phase,metabolic_condition,methylation_frequency_TK2122_rep1,mC_count_TK2122_rep1,coverage_TK2122_rep1,methylation_frequency_TK2122_rep2,mC_count_TK2122_rep2,⋯,amino_acid_sequence,amino_acid_ID,local_41bp_predicted_fold,m5C_position_fold,MFE,associated_TSS_id,TSS_direction,TSS_description,total_annotations,alternate_annotations
,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>,<int>,<dbl>,<int>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>
1,2023350,2024341,+,stationary,S,0.00,0,565,0.00,0,⋯,.,.,|||||||....|||||||||........||||...||||||,single_stranded,-224.51,.,.,.,1,NA
2,1027413,1028404,+,stationary,S,0.00,3,884,0.00,5,⋯,CCG,P,|||..|..|||||..................|||||..|||,single_stranded,-117.74,.,.,.,1,NA
3,2072134,2073766,+,stationary,S,0.00,1,1030,0.00,0,⋯,GCG,A,|.........||||||||||........|||||......||,single_stranded,-90.0,.,.,.,1,NA
4,1756577,1757568,-,stationary,S,0.00,1,992,0.00,0,⋯,CCG,P,||||||.||||||....................||||||..,single_stranded,-86.42,.,.,.,1,NA
5,612640,613631,+,stationary,S,0.29,101,346,0.24,147,⋯,GCG,A,||||||....|||||||||||||..||||||||||..||||,base_paired,-80.41,.,.,.,1,NA


In [18]:
###strain TK2241

defaultW <- getOption("warn")
options(warn = -1)


#load CGmaps into session
TK2241statS_CGmap_rep1 <-read.delim("../cgmaps/TK2241statS_totalRNA_Dec2016.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK2241", growth_phase='stationary', metabolic_condition="S", replicate="rep1")

TK2241statS_CGmap_rep2 <-read.delim("../cgmaps/TK2241statS_totalRNA_Oct2020.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK2241", growth_phase='stationary', metabolic_condition="S", replicate="rep2")

TK2241statS_CGmap_rep3 <-read.delim("../cgmaps/TK2241_statS_totalRNA_rep3.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK2241", growth_phase='stationary', metabolic_condition="S", replicate="rep3")



# process raw CGmaps
TK2241statS_rep1_df <- processCGmap(TK2241statS_CGmap_rep1)
TK2241statS_rep2_df <- processCGmap(TK2241statS_CGmap_rep2)
TK2241statS_rep3_df <- processCGmap(TK2241statS_CGmap_rep3)


#merge reps8
TK2241statS <- merge_del_3reps(x=TK2241statS_rep1_df, y=TK2241statS_rep2_df,z=TK2241statS_rep3_df, strain='TK2241')

# get quantiles
"quantiles:"
get_quantiles(strain_cgmap = TK2241statS, strain="TK2241", reps=3, min_cov = 47)

#compare methylation frequencies & annotate
TK2241statS_anal <- CompareCGmap_3reps(strain_cgmap = TK2241statS, 
                                TS559_cgmap = TS559statS, 
                                strain="TK2241", 
                                annotation = annotation,
                                min_cov = 47)

write.table(TK2241statS_anal, file = "../processed_cgmaps/TK2241statS_annotated", sep = "\t", row.names = F)

#enumerate gains & losses
"losses & gains:"
nrow( TK2241statS_anal %>% filter(log2FC < 0) )
nrow( TK2241statS_anal %>% filter(log2FC > 0) )


"absolute losses & gains:"
nrow( TK2241statS_anal %>% filter(difference == "ABS LOSS") )
nrow( TK2241statS_anal %>% filter(difference == "ABS GAIN") )


# TK2241statS_cat <- enumerate_hiconf_2reps(strain_cgmap = TK2241statS, strain="TK2241",min_cov = 47)

Sys.sleep(15)


[1] "quantiles:"

[1] 18  9  8

In [11]:
###strain TK2304

defaultW <- getOption("warn")
options(warn = -1)


#load CGmaps into session
TK2304statS_CGmap_rep1 <-read.delim("../cgmaps/TK2304statS_totalRNA_Dec2016.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK2304", growth_phase='stationary', metabolic_condition="S", replicate="rep1")

TK2304statS_CGmap_rep2 <-read.delim("../cgmaps/TK2304statS_totalRNA_Oct2020.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK2304", growth_phase='stationary', metabolic_condition="S", replicate="rep2")

# process raw CGmaps
TK2304statS_rep1_df <- processCGmap(TK2304statS_CGmap_rep1)
TK2304statS_rep2_df <- processCGmap(TK2304statS_CGmap_rep2)

#merge reps
TK2304statS <- merge_del_reps(x=TK2304statS_rep1_df, y=TK2304statS_rep2_df, strain='TK2304')

# get quantiles
"quantiles:"
get_quantiles(strain_cgmap = TK2304statS, strain="TK2304", min_cov = 47)

#compare methylation frequencies & annotate
TK2304statS_anal <- CompareCGmap(strain_cgmap = TK2304statS, 
                                TS559_cgmap = TS559statS, 
                                strain="TK2304", 
                                annotation = annotation,
                                min_cov = 47)

write.table(TK2304statS_anal, file = "../processed_cgmaps/TK2304statS_annotated", sep = "\t", row.names = F)

#enumerate gains & losses
"losses & gains:"
nrow( TK2304statS_anal %>% filter(log2FC < 0) )
nrow( TK2304statS_anal %>% filter(log2FC > 0) )


"absolute losses & gains:"
nrow( TK2304statS_anal %>% filter(difference == "ABS LOSS") )
nrow( TK2304statS_anal %>% filter(difference == "ABS GAIN") )

head(TK2304statS_anal)

# TK2304statS_cat <- enumerate_hiconf_2reps(strain_cgmap = TK2304statS, strain="TK2304",min_cov = 47)

Sys.sleep(10)

[1] "quantiles:"

[1] 5 5

[1] "losses & gains:"

[1] 6

[1] 0

[1] "absolute losses & gains:"

[1] 6

[1] 0

,TS559_position,KOD1_position,strand,growth_phase,metabolic_condition,methylation_frequency_TK2304_rep1,mC_count_TK2304_rep1,coverage_TK2304_rep1,methylation_frequency_TK2304_rep2,mC_count_TK2304_rep2,⋯,amino_acid_sequence,amino_acid_ID,local_41bp_predicted_fold,m5C_position_fold,MFE,associated_TSS_id,TSS_direction,TSS_description,total_annotations,alternate_annotations
,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>,<int>,<dbl>,<int>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>
1,2025555,2026546,+,stationary,S,0.00,2,3925,0.00,0,⋯,.,.,...||||||||....||||||...|||||||.......|||,base_paired,-440.69,.,.,.,1,NA
2,2025624,2026615,+,stationary,S,0.00,3,1674,0.00,0,⋯,.,.,..||||||.................................,single_stranded,-440.69,.,.,.,1,NA
3,2025625,2026616,+,stationary,S,0.00,1,1675,0.00,2,⋯,.,.,.||||||..................................,single_stranded,-440.69,.,.,.,1,NA
4,2025645,2026636,+,stationary,S,0.00,1,2289,0.00,0,⋯,.,.,.......................||||||.....||||||.,single_stranded,-440.69,.,.,.,1,NA
5,2023357,2024348,+,stationary,S,0.02,3,128,0.01,3,⋯,.,.,....|||||||||........||||...||||||-------,single_stranded,-224.51,.,.,.,1,NA
6,1728374,1729365,+,stationary,S,0.00,0,133,0.00,0,⋯,CUU,L,...........||||||..|||.....|||||.........,base_paired,-20.17,.,.,.,1,NA


In [20]:
# ###strain TK2045Ndel

# defaultW <- getOption("warn")
# options(warn = -1)


# #load CGmaps into session
# TK2045NdelstatS_CGmap_rep1 <-read.delim("../cgmaps/TK2045Ndel_statS_totalRNA_rep1.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
#     mutate(strain="TK2045Ndel", growth_phase='stationary', metabolic_condition="S", replicate="rep1")

# TK2045NdelstatS_CGmap_rep2 <-read.delim("../cgmaps/TK2045Ndel_statS_totalRNA_rep2.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
#     mutate(strain="TK2045Ndel", growth_phase='stationary', metabolic_condition="S", replicate="rep2")

# # process raw CGmaps
# TK2045NdelstatS_rep1_df <- processCGmap(TK2045NdelstatS_CGmap_rep1)
# TK2045NdelstatS_rep2_df <- processCGmap(TK2045NdelstatS_CGmap_rep2)

# #merge reps
# TK2045NdelstatS <- merge_del_reps(x=TK2045NdelstatS_rep1_df, y=TK2045NdelstatS_rep2_df, strain='TK2045Ndel')

# # get quantiles
# "quantiles:"
# get_quantiles(strain_cgmap = TK2045NdelstatS, strain="TK2045Ndel", min_cov = 47)

# #compare methylation frequencies & annotate
# TK2045NdelstatS_anal <- CompareCGmap(strain_cgmap = TK2045NdelstatS, 
#                                 TS559_cgmap = TS559statS, 
#                                 strain="TK2045Ndel", 
#                                 annotation = annotation,
#                                 min_cov = 47)

# write.table(TK2045NdelstatS_anal, file = "../processed_cgmaps/TK2045NdelstatS_annotated", sep = "\t", row.names = F)

# #enumerate gains & losses
# "losses & gains:"
# nrow( TK2045NdelstatS_anal %>% filter(log2FC < 0) )
# nrow( TK2045NdelstatS_anal %>% filter(log2FC > 0) )

# "absolute losses & gains:"
# nrow( TK2045NdelstatS_anal %>% filter(difference == "ABS LOSS") )
# nrow( TK2045NdelstatS_anal %>% filter(difference == "ABS GAIN") )

# head(TK2045NdelstatS_anal)

# TK2045NdelstatS_cat <- enumerate_hiconf_2reps(strain_cgmap = TK2045NdelstatS, strain="TK2045Ndel",min_cov = 47)

# Sys.sleep(10)

In [21]:
###strain TK0234_0224

defaultW <- getOption("warn")
options(warn = -1)


#load CGmaps into session
TK0234_0224statS_CGmap_rep1 <-read.delim("../cgmaps/TK0234_0224_statS_totalRNA_rep1.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK0234_0224", growth_phase='stationary', metabolic_condition="S", replicate="rep1")

TK0234_0224statS_CGmap_rep2 <-read.delim("../cgmaps/TK0234_0224_statS_totalRNA_rep2.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK0234_0224", growth_phase='stationary', metabolic_condition="S", replicate="rep2")

# process raw CGmaps
TK0234_0224statS_rep1_df <- processCGmap(TK0234_0224statS_CGmap_rep1)
TK0234_0224statS_rep2_df <- processCGmap(TK0234_0224statS_CGmap_rep2)

#merge reps
TK0234_0224statS <- merge_del_reps(x=TK0234_0224statS_rep1_df, y=TK0234_0224statS_rep2_df, strain='TK0234_0224')

# get quantiles
"quantiles:"
get_quantiles(strain_cgmap = TK0234_0224statS, strain="TK0234_0224", min_cov = 47)

#compare methylation frequencies & annotate
TK0234_0224statS_anal <- CompareCGmap(strain_cgmap = TK0234_0224statS, 
                                TS559_cgmap = TS559statS, 
                                strain="TK0234_0224", 
                                annotation = annotation,
                                min_cov = 47)

write.table(TK0234_0224statS_anal, file = "../processed_cgmaps/TK0234_0224statS_annotated", sep = "\t", row.names = F)

#enumerate gains & losses
"losses & gains:"
nrow( TK0234_0224statS_anal %>% filter(log2FC < 0) )
nrow( TK0234_0224statS_anal %>% filter(log2FC > 0) )

"absolute losses & gains:"
nrow( TK0234_0224statS_anal %>% filter(difference == "ABS LOSS") )
nrow( TK0234_0224statS_anal %>% filter(difference == "ABS GAIN") )

head(TK0234_0224statS_anal)

# TK0234_0224statS_cat <- enumerate_hiconf_2reps(strain_cgmap = TK0234_0224statS, strain="TK0234_0224",min_cov = 47)

Sys.sleep(10)

[1] "quantiles:"

[1] 7 9

In [22]:
###strain TK0234_0729

defaultW <- getOption("warn")
options(warn = -1)


#load CGmaps into session
TK0234_0729statS_CGmap_rep1 <-read.delim("../cgmaps/TK0234_0729_statS_totalRNA_rep1.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK0234_0729", growth_phase='stationary', metabolic_condition="S", replicate="rep1")

TK0234_0729statS_CGmap_rep2 <-read.delim("../cgmaps/TK0234_0729_statS_totalRNA_rep2.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK0234_0729", growth_phase='stationary', metabolic_condition="S", replicate="rep2")

# process raw CGmaps
TK0234_0729statS_rep1_df <- processCGmap(TK0234_0729statS_CGmap_rep1)
TK0234_0729statS_rep2_df <- processCGmap(TK0234_0729statS_CGmap_rep2)

#merge reps
TK0234_0729statS <- merge_del_reps(x=TK0234_0729statS_rep1_df, y=TK0234_0729statS_rep2_df, strain='TK0234_0729')

# get quantiles
"quantiles:"
get_quantiles(strain_cgmap = TK0234_0729statS, strain="TK0234_0729", min_cov = 47)

#compare methy8lation frequencies & annotate
TK0234_0729statS_anal <- CompareCGmap(strain_cgmap = TK0234_0729statS, 
                                TS559_cgmap = TS559statS, 
                                strain="TK0234_0729", 
                                annotation = annotation,
                                min_cov = 47)

write.table(TK0234_0729statS_anal, file = "../processed_cgmaps/TK0234_0729statS_annotated", sep = "\t", row.names = F)

#enumerate gains & losses
"losses & gains:"
nrow( TK0234_0729statS_anal %>% filter(log2FC < 0) )
nrow( TK0234_0729statS_anal %>% filter(log2FC > 0) )

"absolute losses & gains:"
nrow( TK0234_0729statS_anal %>% filter(difference == "ABS LOSS") )
nrow( TK0234_0729statS_anal %>% filter(difference == "ABS GAIN") )

head(TK0234_0729statS_anal)

# TK0234_0729statS_cat <- enumerate_hiconf_2reps(strain_cgmap = TK0234_0729statS, strain="TK0234_0729",min_cov = 47)

Sys.sleep(15)

[1] "quantiles:"

[1] 7 9

In [12]:
###strain TK2304_1935

defaultW <- getOption("warn")
options(warn = -1)


#load CGmaps into session
TK2304_1935statS_CGmap_rep1 <-read.delim("../cgmaps/TK2304_1935_statS_totalRNA_rep1.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK2304_1935", growth_phase='stationary', metabolic_condition="S", replicate="rep1")

TK2304_1935statS_CGmap_rep2 <-read.delim("../cgmaps/TK2304_1935_statS_totalRNA_rep2.CGmap", sep='\t', header=FALSE,col.names=cols) %>%
    mutate(strain="TK2304_1935", growth_phase='stationary', metabolic_condition="S", replicate="rep2")

# process raw CGmaps
TK2304_1935statS_rep1_df <- processCGmap(TK2304_1935statS_CGmap_rep1)
TK2304_1935statS_rep2_df <- processCGmap(TK2304_1935statS_CGmap_rep2)

#merge reps
TK2304_1935statS <- merge_del_reps(x=TK2304_1935statS_rep1_df, y=TK2304_1935statS_rep2_df, strain='TK2304_1935')

# get quantiles
"quantiles:"
get_quantiles(strain_cgmap = TK2304_1935statS, strain="TK2304_1935", min_cov = 47)

#compare methylation frequencies & annotate
TK2304_1935statS_anal <- CompareCGmap(strain_cgmap = TK2304_1935statS, 
                                TS559_cgmap = TS559statS, 
                                strain="TK2304_1935", 
                                annotation = annotation,
                                min_cov = 47)

write.table(TK2304_1935statS_anal, file = "../processed_cgmaps/TK2304_1935statS_annotated", sep = "\t", row.names = F)

#enumerate gains & losses
"losses & gains:"
nrow( TK2304_1935statS_anal %>% filter(log2FC < 0) )
nrow( TK2304_1935statS_anal %>% filter(log2FC > 0) )

"absolute losses & gains:"
nrow( TK2304_1935statS_anal %>% filter(difference == "ABS LOSS") )
nrow( TK2304_1935statS_anal %>% filter(difference == "ABS GAIN") )

head(TK2304_1935statS_anal)

# TK2304_1935statS_cat <- enumerate_hiconf_2reps(strain_cgmap = TK2304_1935statS, strain="TK2304_1935",min_cov = 47)

Sys.sleep(15)

[1] "quantiles:"

[1] 7 6

[1] "losses & gains:"

[1] 17

[1] 6

[1] "absolute losses & gains:"

[1] 17

[1] 5

,TS559_position,KOD1_position,strand,growth_phase,metabolic_condition,methylation_frequency_TK2304_1935_rep1,mC_count_TK2304_1935_rep1,coverage_TK2304_1935_rep1,methylation_frequency_TK2304_1935_rep2,mC_count_TK2304_1935_rep2,⋯,amino_acid_sequence,amino_acid_ID,local_41bp_predicted_fold,m5C_position_fold,MFE,associated_TSS_id,TSS_direction,TSS_description,total_annotations,alternate_annotations
,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>,<int>,<dbl>,<int>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>
1,1385895,1386886,-,stationary,S,0.05,218,4057,0.05,165,⋯,ACC,T,|......||||||................|..|..||||..,single_stranded,-134.15,.,.,.,1,NA
2,578337,579328,+,stationary,S,0.05,88,1641,0.04,66,⋯,.,.,.,NA,.,TTS0579315_+,+,primary,1,NA
3,2026306,2027297,+,stationary,S,0.06,73,1324,0.06,81,⋯,.,.,....|.......||||....|||...|||||||||...|||,base_paired,-440.69,.,.,.,1,NA
4,931389,932380,-,stationary,S,0.13,53,421,0.08,41,⋯,GCC,A,----.............................|||||||.,single_stranded,-2.42,TTS0932396_-,-,primary,1,NA
5,461434,462425,+,stationary,S,0.10,52,541,0.06,51,⋯,.,.,.,NA,.,.,.,.,0,NA
6,2025555,2026546,+,stationary,S,0.00,2,4988,0.00,1,⋯,.,.,...||||||||....||||||...|||||||.......|||,base_paired,-440.69,.,.,.,1,NA


# CAT TABLE

In [26]:
cat_statS_data <- merge(x=TS559statS_cat, y = TK0224statS_cat, by='TS559_position', all=TRUE) %>%
    merge(y = TK0234statS_cat, by='TS559_position', all=TRUE) %>%
    merge(y = TK0704statS_cat, by='TS559_position', all=TRUE) %>%
    merge(y = TK0729statS_cat, by='TS559_position', all=TRUE) %>%
    merge(y = TK0872statS_cat, by='TS559_position', all=TRUE) %>%
    merge(y = TK1273statS_cat, by='TS559_position', all=TRUE) %>%
    merge(y = TK1917statS_cat, by='TS559_position', all=TRUE) %>%
    merge(y = TK1935statS_cat, by='TS559_position', all=TRUE) %>%
    merge(y = TK2122statS_cat, by='TS559_position', all=TRUE) %>%
    merge(y = TK2241statS_cat, by='TS559_position', all=TRUE) %>%
    merge(y = TK2304statS_cat, by='TS559_position', all=TRUE) %>%
    merge(y = TK0234_0224statS_cat, by='TS559_position', all=TRUE) %>%
    merge(y = TK0234_0729statS_cat, by='TS559_position', all=TRUE) %>%
    merge(y = TK2304_1935statS_cat, by='TS559_position', all=TRUE) %>%
    #merge(y = TK2045statS_cat, by='TS559_position', all=TRUE) %>%
    #merge(y = TK2045NdelstatS_cat, by='TS559_position', all=TRUE) %>%

    filter(TS559_hiconf==TRUE |
    TK0224_hiconf==TRUE | 
    TK0234_hiconf==TRUE | 
    TK0704_hiconf==TRUE | 
    TK0729_hiconf==TRUE | 
    TK0872_hiconf==TRUE | 
    TK1273_hiconf==TRUE | 
    TK1917_hiconf==TRUE | 
    TK1935_hiconf==TRUE | 
    TK2122_hiconf==TRUE | 
    TK2241_hiconf==TRUE | 
    TK2304_hiconf==TRUE |
    TK0234_0224_hiconf==TRUE |
    TK0234_0729_hiconf==TRUE |
    TK2304_1935_hiconf==TRUE  
           
    )

#write.table(cat_statS_data, file = "../processed_cgmaps/cat_statS_data", sep = "\t", row.names = F)
nrow(cat_statS_data)
head(cat_statS_data)

[1] 1272

,TS559_position,freq_TS559_rep1,m5C_cov/total_cov_TS559_rep1,freq_TS559_rep2,m5C_cov/total_cov_TS559_rep2,freq_TS559_rep3,m5C_cov/total_cov_TS559_rep3,TS559_hiconf,freq_TK0224_rep1,m5C_cov/total_cov_TK0224_rep1,⋯,freq_TK0234_0729_rep1,m5C_cov/total_cov_TK0234_0729_rep1,freq_TK0234_0729_rep2,m5C_cov/total_cov_TK0234_0729_rep2,TK0234_0729_hiconf,freq_TK2304_1935_rep1,m5C_cov/total_cov_TK2304_1935_rep1,freq_TK2304_1935_rep2,m5C_cov/total_cov_TK2304_1935_rep2,TK2304_1935_hiconf
,<int>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<lgl>,<dbl>,<chr>,⋯,<dbl>,<chr>,<dbl>,<chr>,<lgl>,<dbl>,<chr>,<dbl>,<chr>,<lgl>
1,6370,0.08,2 / 25,0.08,8 / 99,0.01,2 / 255,FALSE,0.07,2 / 27,⋯,0.02,5 / 219,0.03,9 / 276,FALSE,0.00,0 / 291,0.00,0 / 359,FALSE
2,12052,0.00,0 / 41,0.04,8 / 214,0.06,15 / 245,FALSE,0.03,1 / 35,⋯,0.04,13 / 312,0.06,16 / 258,FALSE,0.06,17 / 263,0.03,10 / 346,FALSE
3,19557,0.06,3 / 47,0.19,35 / 189,0.08,37 / 452,FALSE,0.11,8 / 71,⋯,0.14,50 / 350,0.16,81 / 505,TRUE,0.05,17 / 310,0.08,31 / 382,FALSE
4,22822,0.01,3 / 317,0.05,41 / 868,0.01,18 / 1860,FALSE,0.01,3 / 336,⋯,0.02,34 / 1561,0.04,68 / 1595,FALSE,0.01,12 / 1809,0.01,25 / 1925,FALSE
5,23107,0.02,1 / 63,0.05,9 / 176,0.00,0 / 475,FALSE,0.02,1 / 62,⋯,0.00,0 / 275,0.00,0 / 308,FALSE,0.00,0 / 298,0.00,0 / 332,FALSE
6,23579,0.09,7 / 79,0.07,14 / 197,0.00,0 / 306,FALSE,0.01,1 / 89,⋯,0.00,0 / 223,0.00,1 / 229,FALSE,0.00,1 / 251,0.00,1 / 269,FALSE
